In [ ]:
%pip install opencv-python
%pip install streamlit
%pip install numpy pandas matplotlib
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Detect Image

In [2]:
import cv2
import time
import matplotlib.pyplot as plt
from ultralytics import YOLO


In [ ]:
# Load YOLOv8s model
model = YOLO('yolov8s.pt')

# Set the correct image path
image_path = 'assets/test/people1.jpg'

# Read the image
image = cv2.imread(image_path)

# Start timer
start_time = time.time()

# Run detection with confidence threshold and only for people (class 0)
results = model.predict(image, classes=[0], conf=0.35)

# End timer
end_time = time.time()
inference_time = end_time - start_time

# Count number of detected persons
count = len(results[0].boxes.cls)

# Draw boxes around detected people
for box in results[0].boxes.xyxy:
    x1, y1, x2, y2 = map(int, box.tolist())
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, "Person", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# Add people count and inference time on image
cv2.putText(image, f"People Count: {count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
cv2.putText(image, f"Time: {inference_time:.2f}s", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

# Show the image
cv2.imshow('Detected People', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(f"✅ People detected: {count}")
print(f"⏱️ Inference time: {end_time - start_time:.2f} seconds")


0: 384x640 12 persons, 266.4ms
Speed: 3.0ms preprocess, 266.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
✅ People detected: 12
⏱️ Inference time: 0.46 seconds


Detect Video

In [12]:
def detect_people_video():
    # Start total time
    total_start = time.time()
    
    # Load the model
    model = YOLO('yolov8s.pt')
    
    # Paths
    video_path = 'assets/test/The CCTV People Demo 2.mp4'  # Input video
    output_path = 'assets/result/output_video.mp4'           # Output video

    # Read the video
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run detection with confidence threshold
        results = model.predict(frame, classes=[0], conf=0.35)
        
        count = len(results[0].boxes.cls)  # Count detected people
        
        for box in results[0].boxes.xyxy:
            x1, y1, x2, y2 = box.tolist()
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        
        # Put people count on the video frame
        cv2.putText(frame, f"People: {count}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        # Save the frame to output video
        out.write(frame)
    
    # Release everything
    cap.release()
    out.release()
    
    # End total time
    total_end = time.time()
    total_time = total_end - total_start

    print(f"✅ Video saved to: {output_path}")
    print(f"⏱️ Total processing time: {total_time:.2f} seconds")
    
    return output_path

# Call the function
output_video = detect_people_video()


0: 384x640 (no detections), 166.1ms
Speed: 2.7ms preprocess, 166.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 174.3ms
Speed: 1.7ms preprocess, 174.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 179.4ms
Speed: 2.5ms preprocess, 179.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 154.9ms
Speed: 2.8ms preprocess, 154.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 145.5ms
Speed: 2.2ms preprocess, 145.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 150.0ms
Speed: 1.8ms preprocess, 150.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 148.1ms
Speed: 1.6ms preprocess, 148.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 140.0ms
Speed: 2.2ms prepr

Detect Real Time Camera

In [15]:
def detect_people_camera():
    # Load YOLO model
    model = YOLO('yolov8s.pt')

    # Open webcam
    cap = cv2.VideoCapture(1)  # 0 = default webcam

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture from camera.")
            break
        
        # Run detection
        results = model.predict(frame, classes=[0], conf=0.35)  # Detect person only
        
        # People count
        count = len(results[0].boxes.cls)
        
        # Draw boxes
        for box in results[0].boxes.xyxy:
            x1, y1, x2, y2 = box.tolist()
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        
        # Display people count
        cv2.putText(frame, f"People: {count}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        # Show frame in OpenCV window
        cv2.imshow('Real-time People Detection', frame)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release webcam and close windows
    cap.release()
    cv2.destroyAllWindows()

# Call the function
detect_people_camera()


0: 480x640 (no detections), 195.3ms
Speed: 3.1ms preprocess, 195.3ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 260.6ms
Speed: 6.0ms preprocess, 260.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 197.3ms
Speed: 2.2ms preprocess, 197.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 188.2ms
Speed: 2.7ms preprocess, 188.2ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 190.2ms
Speed: 2.2ms preprocess, 190.2ms inference, 2.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 187.8ms
Speed: 2.3ms preprocess, 187.8ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 178.9ms
Speed: 1.9ms preprocess, 178.9ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 194.5ms
Speed: 2.9ms prepr